In [2]:
import sys,subprocess
pkgs=["autogluon.tabular==1.4.0","catboost==1.2.8","tqdm"]
subprocess.check_call([sys.executable,"-m","pip","install","-q"]+pkgs)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 12.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


0

In [26]:
import pandas as pd
import numpy as np
from pathlib import Path
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from catboost import CatBoostRegressor,Pool
from catboost.utils import get_gpu_device_count
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from pathlib import Path
import pandas as pd

# ------------------
# Конфиг
# ------------------
TRAIN_CSV = "/kaggle/input/avito-vseros/train_dataset.parquet"
TEST_CSV  = "/kaggle/input/avito-vseros/test_dataset.parquet"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

SEED = 19

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)


Using device: cuda


In [30]:
train_df = pd.read_parquet(TRAIN_CSV).drop(['ID'], axis=1)
test_df = pd.read_parquet(TEST_CSV).drop(['ID'], axis=1)

sample = pd.read_csv('/kaggle/input/avito-vseros/sample_submission(1).csv')

In [46]:
target_col = 'price_TARGET'
feature_cols = [c for c in train_df.columns if c not in [target_col]]


categorical_cols = [c for c in feature_cols if train_df[c].dtype == 'object' or train_df[c].dtype.name == 'category']

for col in categorical_cols:
    train_df[col] = train_df[col].astype(str).fillna('nan')
    test_df[col] = test_df[col].astype(str).fillna('nan')

In [47]:
X = train_df.drop(['price_TARGET'], axis=1)
y = train_df['price_TARGET'].values

X_test = test_df.copy()

In [50]:
train_df.shape

(70000, 34)

In [49]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=True,
)

X_tr_feat = fg.fit_transform(X=X, y=y)
X_te_feat = fg.transform(X_test)

print(X_tr_feat.shape, X_te_feat.shape)
print(X_tr_feat.columns[:30])

(70000, 399) (25000, 399)
Index(['doors_number', 'steering_wheel', 'mileage', 'latitude', 'longitude',
       'audiosistema_mult', 'shini_i_diski_mult',
       'aktivnaya_bezopasnost_mult_raw_text',
       'multimedia_navigacia_mult_raw_text',
       'pomosh_pri_vozhdenii_mult_raw_text', 'equipment', 'body_type',
       'drive_type', 'engine_type', 'color', 'pts', 'audiosistema', 'diski',
       'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom',
       'usilitel_rul', 'crashes_count', 'owners_count',
       'aktivnaya_bezopasnost_mult', 'electroprivod_mult', 'fary_mult',
       'multimedia_navigacia_mult', 'obogrev_mult'],
      dtype='object')


In [53]:
import numpy as np

cat_cols = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("category", "object")
]

for c in cat_cols:
    X_tr_feat[c] = X_tr_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")
    if c in X_te_feat.columns:
        X_te_feat[c] = X_te_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")

print("Категориальных колонок:", len(cat_cols))

Категориальных колонок: 29


In [54]:
cat_features = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("object",)  # теперь всё строковое
]
print(len(cat_features), "cat_features")

29 cat_features


In [55]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import numpy as np

idx = np.arange(len(X_tr_feat))
np.random.seed(42)
np.random.shuffle(idx)
small_idx = idx[:1000]

X_small = X_tr_feat
y_small = y

train_pool = Pool(X_small, y_small, cat_features=cat_features)

model_gpu_test = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=300,
    learning_rate=0.05,
    depth=6,
    task_type="GPU",
    devices="0",
    verbose=50,
    random_seed=42,
)

print("=== GPU sanity fit (300 iters on 1000 rows) ===")
model_gpu_test.fit(train_pool)

=== GPU sanity fit (300 iters on 1000 rows) ===
0:	learn: 888883.2387011	total: 223ms	remaining: 1m 6s
50:	learn: 547622.7923800	total: 1.68s	remaining: 8.22s
100:	learn: 501367.1508686	total: 3.01s	remaining: 5.92s
150:	learn: 484522.0255374	total: 4.37s	remaining: 4.31s
200:	learn: 472820.6666735	total: 5.76s	remaining: 2.84s
250:	learn: 463052.0404265	total: 7.11s	remaining: 1.39s
299:	learn: 456213.3132158	total: 8.47s	remaining: 0us


In [56]:
test_pred = model_gpu_test.predict(X_te_feat)



In [58]:
sample['target'] = test_pred

In [59]:
sample.to_csv('subbbb.csv', index=False)